In [ ]:
# Download the Spacy Model
# !python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load("en_core_web_md")

In [8]:
word1 = nlp("king")
word2 = nlp("queen")

similarity_score = word1.similarity(word2)

print(f"Similarity between '{word1}' and '{word2}': {similarity_score:.4f}")

Similarity between 'king' and 'queen': 0.7253


# Evaluation

In [ ]:
# import pandas as pd
# from scipy.stats import pearsonr, spearmanr

# def evaluate_spacy_model(test_dataset_path):
#     # Load the spaCy model
#     nlp = spacy.load("en_core_web_md")

#     # Load the test dataset
#     test_data = pd.read_csv(test_dataset_path, delimiter="\t")

#     # Extract word pairs and human similarity scores
#     word_pairs = test_data[['word1', 'word2']]
#     human_scores = test_data['SimLex999']

#     # Compute spaCy-predicted similarity scores
#     predicted_scores = []
#     for _, row in word_pairs.iterrows():
#         word1_vec = nlp(row['word1'])
#         word2_vec = nlp(row['word2'])
#         predicted_scores.append((word1_vec.similarity(word2_vec))*10)

#     # Calculate Pearson and Spearman correlations
#     pearson_corr, _ = pearsonr(human_scores, predicted_scores)
#     spearman_corr, _ = spearmanr(human_scores, predicted_scores)

#     return {
#         "Pearson Correlation": pearson_corr,
#         "Spearman Correlation": spearman_corr
#     }

# test_dataset_path = "./datasets/SimLex-999.txt"
# evaluation_results = evaluate_spacy_model(test_dataset_path)
# print("Evaluation Results:")
# print(f"Pearson Correlation: {evaluation_results['Pearson Correlation']:.4f}")
# print(f"Spearman Correlation: {evaluation_results['Spearman Correlation']:.4f}")


Evaluation Results:
Pearson Correlation: 0.4367
Spearman Correlation: 0.4083


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm

def evaluate_spacy_model(test_dataset_path, threshold=0.5):
    test_data = pd.read_csv(test_dataset_path, delimiter="\t")

    word_pairs = test_data[['word1', 'word2']]
    human_scores = test_data['SimLex999']

    predicted_scores = []
    for _, row in tqdm(word_pairs.iterrows(), desc="Evaluating", total=len(word_pairs)):
        word1_vec = nlp(row['word1'])
        word2_vec = nlp(row['word2'])
        similarity = word1_vec.similarity(word2_vec) * 10
        predicted_scores.append(similarity)
    
    predicted_scores = np.array(predicted_scores)
    
    # Normalize scores to 0-1 range
    human_scores_norm = (human_scores - human_scores.min()) / (human_scores.max() - human_scores.min())
    predicted_scores_norm = (predicted_scores - predicted_scores.min()) / (predicted_scores.max() - predicted_scores.min())
    
    pearson_corr, _ = pearsonr(human_scores, predicted_scores)
    spearman_corr, _ = spearmanr(human_scores, predicted_scores)
    correlation = (pearson_corr + spearman_corr) / 2   

    # Continous -> binary values
    human_binary = (human_scores_norm >= threshold).astype(int)
    predicted_binary = (predicted_scores_norm >= threshold).astype(int)
    
    accuracy = accuracy_score(human_binary, predicted_binary)
    precision, recall, f1, _ = precision_recall_fscore_support(human_binary, predicted_binary, average='binary')
    
    return {
        'accuracy': round(accuracy, 4),
        'precision': round(precision, 4),
        'f1_score': round(f1, 4),
        'recall': round(recall, 4),
        'correlation': round(correlation, 4)
    }

# Example usage:
test_dataset_path = "./datasets/SimLex-999.txt"
results = evaluate_spacy_model(test_dataset_path)
for metric, value in results.items():
    print(f"{metric}: {value}")

Evaluating: 100%|██████████| 999/999 [00:07<00:00, 135.79it/s]

accuracy: 0.5816
precision: 0.5401
f1_score: 0.4258
recall: 0.3515
correlation: 0.2275
